In [3]:
print("hello world")

hello world


In [4]:
import pandas as pd
data_path = '../data/skylab_instagram_datathon_dataset.csv'

data = pd.read_csv(data_path, delimiter=';')  # Specify the delimiter (e.g., comma)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [5]:
print(len(data))

704313


In [7]:
print(data.columns)

Index(['period', 'period_end_date', 'compset_group', 'compset',
       'business_entity_doing_business_as_name', 'legal_entity_name',
       'domicile_country_name', 'ultimate_parent_legal_entity_name',
       'primary_exchange_name', 'calculation_type', 'followers', 'pictures',
       'videos', 'comments', 'likes'],
      dtype='object')


In [9]:
unique_compset_groups = data['compset_group'].unique()
print(unique_compset_groups)

['Luxury & Premium & Mainstream' 'Restaurants' 'Beverages'
 'Apparel Retail' 'Food Products' 'Sporting Goods'
 'Sportswear & Athleisure' 'Beauty & Boutique' 'Building Products'
 'Petcare' 'Toys & Collectibles' 'Mattress' 'Fitness & Exercise'
 'Food Retail' 'Discount Retailers ' 'Outdoor Gear'
 'Dermatology and Orthodontics' 'Entertainment' 'Home Appliances'
 'Study (All Brands)']


In [11]:
unique_compset = data['compset'].unique()
print(unique_compset)

['Soft Luxury' 'Luxury & Premium & Mainstream'
 'US Softlines Analyst Interest List'
 'Global Luxury Analysts Interest List' 'Coffee' 'Restaurants' 'Beverages'
 'Sports drinks' 'Energy drinks' 'Fast Fashion'
 'Omnichannel Apparel Retail' 'Apparel Retail' 'US Department Store'
 'Latam Apparel Retail' 'Food Products' 'Snack Bars' 'Pasta Sauce'
 'Ecommerce Apparel Retail' 'Ecommerce Mainstream Apparel Retail'
 'Sporting Goods' 'Casual Dining' 'Hard Luxury' 'Sportswear & Athleisure'
 'Athleisure' 'Footwear' 'Beauty & Boutique' 'Building Products'
 'Sportswear' 'Petcare' 'Premium Brands' 'QSR' 'Alcohol'
 'Toys & Collectibles' 'Mattress' 'Fitness & Exercise' 'US Discount Store'
 'Ecommerce Luxury Apparel Retail' 'Plant-Based Meat' 'Food Retail'
 'Discount Retailers ' 'Mid-Range Watch & Jewelry' 'Outdoor Gear'
 'Plant-Based Dairy' 'Mainstream Brands' 'Dermatology and Orthodontics'
 'Dermatology' 'Entertainment' 'Workout Gear' 'Yoghurt' 'Home Appliances'
 'Soda' 'Fast Casual' 'Study (All Brand

In [28]:
import csv
import json

# Dictionary to store mapping from compset_group to list of unique compsets
from_str = 'ultimate_parent_legal_entity_name'
to_str = 'legal_entity_name'
print_path = 'ultimate_entity_to_subentities.json'
map_from_to = {}

# Path to your CSV file
csv_file_path = '../data/skylab_instagram_datathon_dataset.csv'

# Open the CSV file
with open(csv_file_path, newline='') as csvfile:
    # Create a CSV reader object
    reader = csv.DictReader(csvfile, delimiter=';')
    
    # Iterate over each row in the CSV file
    for row in reader:
        # Extract compset_group and compset values from the current row
        compset_group = row[from_str]
        compset = row[to_str]
        
        # Check if compset_group is already in the map
        if compset_group in map_from_to:
            # If compset_group is already in the map, append compset to its list
            map_from_to[compset_group].append(compset)
        else:
            # If compset_group is not in the map, create a new list with compset and add it to the map
            map_from_to[compset_group] = [compset]

# Convert the compset lists to sets to remove duplicates, then back to lists
for compset_group, compsets in map_from_to.items():
    map_from_to[compset_group] = list(set(compsets))

# Convert the dictionary to a JSON string
compset_group_json = json.dumps(map_from_to, indent=4)

# Path to write the output JSON file
output_file_path = f'../out/{print_path}'

# Write the JSON string to a file
with open(output_file_path, 'w') as outfile:
    outfile.write(compset_group_json)
